<a href="https://colab.research.google.com/github/Aye-Nyein-Thaw/TensorFlow-Beginner/blob/main/coding-exercise/week5/part1/4_Autosave_with_ModelCheckpoint_callback.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autosave with ModelCheckpoint callback

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

In [ ]:
# Prepare Data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

# Use smaller subset to speed things up
x_train = x_train[:10000]
y_train = y_train[:10000]
x_test = x_test[:10000]
y_test = y_test[:10000]

# Define helper functions
def evaluate(model, x_test, y_test):
    test_loss, test_acc = model.evaluate(x=x_test, y=y_test, verbose=0)
    print('accuracy = {acc:0.2f}%, Loss = {loss:0.2f}'.format(acc = test_acc * 100, loss = test_loss))
    
def get_new_model():
    model = Sequential([
        Conv2D(filters=16, input_shape=(32, 32, 3), kernel_size=(3, 3), 
               activation='relu', name='conv_1'),
        Conv2D(filters=8, kernel_size=(3, 3), activation='relu', name='conv_2'),
        MaxPooling2D(pool_size=(4, 4), name='pool_1'),
        Flatten(name='flatten'),
        Dense(units=32, activation='relu', name='dense_1'),
        Dense(units=10, activation='softmax', name='dense_2')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Model Checkpoint Callback

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# save only weights at every epoch
# overwrites everytime

checkpoint_path = 'model_checkpoints/checkpoint'
checkpoint = ModelCheckpoint(filepath = checkpoint_path,
                            frequency = 'epoch',
                            save_weights_only = True,
                            verbose = 1)

In [ ]:
# save only weights at every epoch
# doesn't overwrite, new file everytime

checkpoint_path = 'model_checkpoints/checkpoint_{epoch:02d}'
epoch_checkpoint = ModelCheckpoint(filepath = checkpoint_path,
                            frequency = 'epoch',
                            save_weights_only = True,
                            verbose = 1)

In [ ]:
# Save best model
# monitor validation accuracy

checkpoint_path = 'model_checkpoints/best_checkpoint'
best_checkpoint = ModelCheckpoint(filepath = checkpoint_path,
                            frequency = 'epoch',
                            save_weights_only = True,
                             
                            save_best_only = True,
                            monitor = 'val_accuracy',
                             
                            verbose = 1)

In [ ]:
# Save entire model
# Best model

checkpoint_path = 'model_checkpoints/best_checkpoint_model'
model_best_checkpoint = ModelCheckpoint(filepath = checkpoint_path,
                            frequency = 'epoch',
                            save_weights_only = False,
                             
                            save_best_only = True,
                            monitor = 'val_accuracy',
                             
                            verbose = 1)

In [ ]:
model = get_new_model()
model.fit(x_train, y_train, epochs = 3, callbacks = [model_best_checkpoint], validation_data = (x_test, y_test))

## Compare two models

In [ ]:
from tensorflow.keras.models import load_model

trained_model = load_model('model_checkpoints/best_checkpoint_model')
new_model = get_new_model()

evaluate(trained_model, x_test, y_test)
evaluate(new_model, x_test, y_test)